In [3]:
import asyncio
import json
import sys

import IPython
from IPython import display
from ipykernel.comm import Comm
import tensorflow as tf
import queue
import threading

from IPython.core.magic import register_cell_magic


command_queue = queue.Queue()
# for i in range(1000):
#     command_queue.put('step')  # DEBUG


class DebuggerCommHandler(object):

    def __init__(self, command_queue):
        self.step_count = 0
        self.queue = command_queue
        self.message = 'Empty message'
        
    def target_func(self, comm, msg):
        @comm.on_msg
        def _on_msg(msg):
#             def threaded_handler():
#                 with open('/tmp/debugger2.log', 'at') as f:
#                     f.write('In _on_msg(): self.step_count = %d\n' % self.step_count)  # DEBUG
            self.step_count += 1
            self.queue.put('step')
            comm.send({'step_count': self.step_count, 'message': self.message})
#             thread = threading.Thread(target=threaded_handler)
#             thread.start()
#             thread.join()

        comm.send({})

    def get_from_queue(self):
        return self.queue.get()
    
    def set_message(self, message):
        self.message = message


comm_handler = DebuggerCommHandler(command_queue)
get_ipython().kernel.comm_manager.register_target(
    'debugger_comm_target', comm_handler.target_func)
        

debugger_data = {
    'code_lines': []
}
   
    
def trace_function(frame, event, arg):
    with open('/tmp/debugger2.log', 'at') as f:
        f.write('%s - %s\n' % (frame.f_code.co_filename, frame.f_code.co_name))
#         print('(%s %s %d)' %
#               (event, frame.f_code.co_name, frame.f_lineno))  # DEBUG

    if frame.f_code.co_name == 'target_func':
        return None
    elif frame.f_code.co_filename.startswith('<ipython-input-'):
        # TODO(cais): Remove the stipulated 3 in ipython-input-3.
        source_line = debugger_data['code_lines'][frame.f_lineno - 1]
        message = '*** %s @ "%s" (%s: %s: Line %d)' % (
            event, source_line, frame.f_code.co_filename,
            frame.f_code.co_name, frame.f_lineno)
        comm_handler.set_message(message)
        print(message)

#         sys.settrace(None)
#         print('Calling get_fromt_queue()')  # DEBUG
#         command_queue.get()
#         input()
#         print('DONE Calling get_fromt_queue()')  # DEBUG
#         sys.settrace(trace_function)

#             comm_handler.get_from_queue()

#             print('DONE Calling get_fromt_queue()')  # DEBUG
        return trace_function
    else:
        return None
        

@register_cell_magic
def debugger_magic(line, cell):
    "my debugger jupyter magic"

    display.display(display.HTML(
        '<div id="extension-div">' +
        '<button id="step-button">Step</button>' +
        '</div>'))
    display.display(display.Javascript(filename='debugger_frontend.js'))

#     code_lines = ['async def async_main():']
#     for line in cell.split('\n'):
#         code_lines.append('    %s' % line)
#     code_lines.append('sys.settrace(trace_function)')
#     code_lines.append('await async_main()')
# #     code_lines.append('print(tf)')  # DEBUG
#     code_lines.append('sys.settrace(None)')
#     revised_code = '\n'.join(code_lines)
#     debugger_data['code_lines'] = code_lines
#     print(revised_code)  # DEBUG

    revised_code = cell
    debugger_data['code_lines'] = cell.split('\n')

    def thread_target():
#         asyncio.sleep(5)
#         print("Hello")
#         print(revised_code)
        
        sys.settrace(trace_function)
        IPython.get_ipython().kernel.do_execute(revised_code, False, allow_stdin=True)
        sys.settrace(None)
        
    thread = threading.Thread(target=thread_target())
    thread.start()
#     thread.join()
    return line, cell

In [5]:
%%debugger_magic
print(1234)



def _add_two(v):
    return v + 2




x = 1
y = 11
z = x + y
z = z * 2
z = _add_two(z ** 2)
print(z)

<IPython.core.display.Javascript object>

*** call @ "print(1234)" (<ipython-input-4-ea25cbb5c77c>: <module>: Line 1)
*** line @ "print(1234)" (<ipython-input-4-ea25cbb5c77c>: <module>: Line 1)
1234
*** return @ "print(1234)" (<ipython-input-4-ea25cbb5c77c>: <module>: Line 1)
*** call @ "def _add_two(v):" (<ipython-input-4-ea25cbb5c77c>: <module>: Line 5)
*** line @ "def _add_two(v):" (<ipython-input-4-ea25cbb5c77c>: <module>: Line 5)
*** return @ "def _add_two(v):" (<ipython-input-4-ea25cbb5c77c>: <module>: Line 5)
*** call @ "x = 1" (<ipython-input-4-ea25cbb5c77c>: <module>: Line 11)
*** line @ "x = 1" (<ipython-input-4-ea25cbb5c77c>: <module>: Line 11)
*** return @ "x = 1" (<ipython-input-4-ea25cbb5c77c>: <module>: Line 11)
*** call @ "y = 11" (<ipython-input-4-ea25cbb5c77c>: <module>: Line 12)
*** line @ "y = 11" (<ipython-input-4-ea25cbb5c77c>: <module>: Line 12)
*** return @ "y = 11" (<ipython-input-4-ea25cbb5c77c>: <module>: Line 12)
*** call @ "z = x + y" (<ipython-input-4-ea25cbb5c77c>: <module>: Line 13)
*** line @ "

('',
 'print(1234)\n\n\n\ndef _add_two(v):\n    return v + 2\n\n\n\n\nx = 1\ny = 11\nz = x + y\nz = z * 2\nz = _add_two(z ** 2)\nprint(z)\n')

In [ ]:
%%debugger_magic

import tensorflow as tf
tf.enable_eager_execution()



def _add_one(v):
    return v + 1


def main():
  a = tf.constant(3.14)
  x = 8
  y = 9
  v = [1, 2, 3, 4, 5, 6, 7]  # Array
  z = x + y
  # Invoke _add_one()
  z = _add_one(z)
  print(z)


main()

In [ ]:
import asyncio
async def fn():
  print('hello')
  await asyncio.sleep(1)
  print('world')

await fn()